### Напоминалка для семинаристов по ревью многопоточного кода

<br />

Можно использовать подход сверху-вниз: от общей картины к деталям.

<br />

**В первую очередь понять верхнуровневую организацию по классам / потокам:**
* какие объекты на каких потоках живут
* когда потоки создаются и умирают

```
SalesmanWidget - UI
SalesmanSolver - UI, Background
```

<br />

**Уровень функций / методов:**
* какие функции / методы на каких потоках зовутся _(например, выписать функции и потоки)_

```
SalesmanWidget::update_status (UI thread)
SalesmanSolver::SalesmanSolver (UI thread)
SalesmanSolver::process_city (Background thread)
```

<br />

**Уровень данных:**
* какие данные используются
* как они синхронизированы
* корректны ли эти синхронизации


```
SalesmanWidget::update_status (UI thread)
    r: thread_status_array // mutex: thread_status_mtx
    r: global_solution     // atomic-relaxed
    
SalesmanSolver::SalesmanSolver (UI thread)
    w: costs // no synchronization

SalesmanSolver::process_city (Background thread)
    r: costs // no synchronization
    
SalesmanSolverState:
    int thread_count; // UI only - ok, read on background but threads are synchronized
    int cities_count; // UI only - ok, read on background but threads are synchronized
```

<br />

**Дополнительно проработать вопросы:**
* нет ли неожиданностей после смерти потоков (use-after-free?)
* корректна ли последовательность вызовов синхронизированных методов? Пример с многопоточной очередью задач:

```c++
if (!queue.empty())
    process_task(queue.pop());
```

* синхронизация на 2 atomics - наиболее вероятно, что гонка
* atomics - relaxed режим для счётчиков

<br />

Thread sanitizer